IMPORTS

In [172]:
import pandas as pd
import datetime as dt
import numpy as np
import ast

BASES

MERGES et BASES

In [173]:
df_title_basics = pd.read_csv('BD/title.basics.tsv.gz', compression = 'gzip', sep = '\t')
df_title_ratings = pd.read_csv('BD/title.ratings.tsv.gz', compression = 'gzip', sep = '\t')

/var/folders/2b/18xblx_n4lv24wz932vjckrm0000gp/T/ipykernel_4339/3795572048.py:1: DtypeWarning: Columns (4) have mixed types. Specify dtype option on import or set low_memory=False.
  df_title_basics = pd.read_csv('BD/title.basics.tsv.gz', compression = 'gzip', sep = '\t')


In [174]:
df1 = pd.merge(df_title_basics,
               df_title_ratings,
               how = 'outer',
               on = 'tconst')

In [175]:
df_title_ratings, df_title_basics = 0, 0

In [176]:
df_title_akas = pd.read_csv('BD/title.akas.tsv.gz', compression = 'gzip', sep = '\t')

In [177]:
df_3 = pd.merge(df1,
                df_title_akas,
                how = 'outer',
                left_on = 'tconst',
                right_on = 'titleId')

In [178]:
df1, df_title_akas = 0, 0

In [179]:
df_tmbd_full = pd.read_csv('BD/tmdb_full.csv')

/var/folders/2b/18xblx_n4lv24wz932vjckrm0000gp/T/ipykernel_4339/1088705580.py:1: DtypeWarning: Columns (24) have mixed types. Specify dtype option on import or set low_memory=False.
  df_tmbd_full = pd.read_csv('BD/tmdb_full.csv')


In [181]:
df_films = pd.merge(df_tmbd_full,
                df_3,
                how = 'outer',
                left_on = 'imdb_id',
                right_on = 'tconst')

In [182]:
df_tmbd_full, df_3 = 0, 0

CLEAN DU MERGE

In [183]:
# DROP CE QUI N'EST PAS FILM

df_films = df_films[(df_films['titleType'] == 'movie')]

COLONNES NUMERIQUES

In [184]:
# On remplit les cellules vides par 0

df_films[['runtimeMinutes', 'budget', 'popularity', 'revenue', 'runtime', 'vote_average', 'vote_count', 'startYear', 'endYear', 'averageRating', 'numVotes']] = df_films[['runtimeMinutes', 'budget', 'popularity', 'revenue', 'runtime', 'vote_average', 'vote_count', 'startYear', 'endYear', 'averageRating', 'numVotes']].fillna(0)

In [185]:
# On remplace les '\\N' par 0

df_films[['runtimeMinutes', 'budget', 'popularity', 'revenue', 'runtime', 'vote_average', 'vote_count', 'startYear', 'endYear', 'averageRating', 'numVotes']] = df_films[['runtimeMinutes', 'budget', 'popularity', 'revenue', 'runtime', 'vote_average', 'vote_count', 'startYear', 'endYear', 'averageRating', 'numVotes']].replace('\\N', 0)

/var/folders/2b/18xblx_n4lv24wz932vjckrm0000gp/T/ipykernel_4339/2320903248.py:3: FutureWarning: Downcasting behavior in `replace` is deprecated and will be removed in a future version. To retain the old behavior, explicitly call `result.infer_objects(copy=False)`. To opt-in to the future behavior, set `pd.set_option('future.no_silent_downcasting', True)`
  df_films[['runtimeMinutes', 'budget', 'popularity', 'revenue', 'runtime', 'vote_average', 'vote_count', 'startYear', 'endYear', 'averageRating', 'numVotes']] = df_films[['runtimeMinutes', 'budget', 'popularity', 'revenue', 'runtime', 'vote_average', 'vote_count', 'startYear', 'endYear', 'averageRating', 'numVotes']].replace('\\N', 0)


In [186]:
# On met ces colonnes au format numérique

df_films[['runtimeMinutes', 'budget', 'popularity', 'revenue', 'runtime', 'vote_average', 'vote_count', 'startYear', 'endYear', 'averageRating', 'numVotes']] = df_films[['runtimeMinutes', 'budget', 'popularity', 'revenue', 'runtime', 'vote_average', 'vote_count', 'startYear', 'endYear', 'averageRating', 'numVotes']].apply(pd.to_numeric)

In [187]:
# On transforme les types de données pour 'release_date'

df_films['release_date'] = pd.to_datetime(df_films['release_date'])
df_films['release_date'] = df_films['release_date'].replace('\\N', 0)
df_films['release_date'] = df_films['release_date'].dt.year

In [188]:
# On transforme les types de données pour 'startYear'

df_films['startYear'] = df_films['startYear'].replace('\\N', 0)
df_films['startYear'] = pd.to_numeric(df_films['startYear'], downcast="integer")

In [189]:
# On crée la colonne year_exact

df_films['year_exact'] = df_films[['release_date', 'startYear']].max(axis = 1)

In [190]:
# On crée la colonne 'Decennie'

df_films['Decennie'] = ''
df_films['year_exact'] = df_films['year_exact'].fillna('0')
df_films['year_exact'] = df_films['year_exact'].replace('nan', '0')
df_films['year_exact'] = df_films['year_exact'].astype(str)

In [191]:
df_films['Decennie'] = df_films['year_exact'].str[:3] + "0's"
df_films['Decennie'] = df_films['Decennie'].replace("0.00's", None)
df_films['year_exact'] = pd.to_numeric(df_films['year_exact'])

RUNTIME

In [192]:
# On met de côté les valeurs nulles

df_null1 = df_films[((df_films['runtimeMinutes'].isna()) & (df_films['runtime'].isna()))]
df_null2 = df_films[((df_films['runtimeMinutes'] == 0) & (df_films['runtime'] == 0))]
df_null3 = df_films[((df_films['runtimeMinutes'] == 0) & (df_films['runtime'].isna()))]
df_null4 = df_films[((df_films['runtimeMinutes'].isna()) & (df_films['runtime'] == 0))]

In [193]:
df_null = pd.concat([df_null1, df_null2, df_null3, df_null4])

In [194]:
df_null1, df_null2, df_null3, df_null4 = 0, 0, 0, 0

In [195]:
# On prend les valeurs utiles

df_utile = df_films[~((df_films['runtimeMinutes'].isna()) & (df_films['runtime'].isna()))]
df_utile = df_utile[~((df_utile['runtimeMinutes'] == 0) & (df_utile['runtime'] == 0))]
df_utile = df_utile[~((df_utile['runtimeMinutes'] == 0) & (df_utile['runtime'].isna()))]
df_utile = df_utile[~((df_utile['runtimeMinutes'].isna()) & (df_utile['runtime'] == 0))]

In [196]:
# Calcul différence de runtime des 2 tables

df_utile['difference_runtime'] = abs(df_utile['runtime'] - df_utile['runtimeMinutes'])

In [197]:
# On met de côté là où les 2 valeurs sont identiques

df_a_garder_1 = df_utile[(df_utile['runtimeMinutes'] == df_utile['runtime'])]

In [198]:
# Création colonne RuntimeExact
df_a_garder_1['runtime_exact'] = df_a_garder_1['runtimeMinutes']

df_a_travailler_1 = df_utile[~(df_utile['runtimeMinutes'] == df_utile['runtime'])]

/var/folders/2b/18xblx_n4lv24wz932vjckrm0000gp/T/ipykernel_4339/3570380868.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_a_garder_1['runtime_exact'] = df_a_garder_1['runtimeMinutes']


In [199]:
# On met de côté là où une valeur manque seulement

df_a_garder_2 = df_a_travailler_1[((df_a_travailler_1['runtimeMinutes'].isna()) | (df_a_travailler_1['runtime'].isna())| (df_a_travailler_1['runtime'] == 0) | (df_a_travailler_1['runtimeMinutes'] == 0))]

In [200]:
# On conserve la donnée existante

df_a_garder_2['runtime_exact'] = df_a_garder_2[['runtimeMinutes', 'runtime']].max(axis = 1)

/var/folders/2b/18xblx_n4lv24wz932vjckrm0000gp/T/ipykernel_4339/3521993277.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_a_garder_2['runtime_exact'] = df_a_garder_2[['runtimeMinutes', 'runtime']].max(axis = 1)


In [201]:
df_a_travailler_2 = df_a_travailler_1[~((df_a_travailler_1['runtimeMinutes'].isna()) | (df_a_travailler_1['runtime'].isna())| (df_a_travailler_1['runtime'] == 0) | (df_a_travailler_1['runtimeMinutes'] == 0))]
df_a_travailler_2['difference_runtime'] = abs(df_a_travailler_2['runtime'] - df_a_travailler_2['runtimeMinutes'])

/var/folders/2b/18xblx_n4lv24wz932vjckrm0000gp/T/ipykernel_4339/1261991755.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_a_travailler_2['difference_runtime'] = abs(df_a_travailler_2['runtime'] - df_a_travailler_2['runtimeMinutes'])


In [202]:
# On met de côté là où la différence de durée est négligeable

df_a_garder_3 = df_a_travailler_2[df_a_travailler_2['difference_runtime'] <= 20]

In [203]:
# On prend comme base la valeur max des 2 colonnes

df_a_garder_3['runtime_exact'] = df_a_garder_3[['runtimeMinutes', 'runtime']].max(axis = 1)

/var/folders/2b/18xblx_n4lv24wz932vjckrm0000gp/T/ipykernel_4339/880983650.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_a_garder_3['runtime_exact'] = df_a_garder_3[['runtimeMinutes', 'runtime']].max(axis = 1)


In [204]:
df_a_travailler_3 = df_a_travailler_2[~(df_a_travailler_2['difference_runtime'] <= 20)]

In [205]:
# Pour le restant des données, on conserve la valeur max des 2 colonnes

df_a_travailler_3['runtime_exact'] = df_a_travailler_3[['runtimeMinutes', 'runtime']].max(axis = 1)
df_a_garder_4 = df_a_travailler_3

/var/folders/2b/18xblx_n4lv24wz932vjckrm0000gp/T/ipykernel_4339/264539281.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_a_travailler_3['runtime_exact'] = df_a_travailler_3[['runtimeMinutes', 'runtime']].max(axis = 1)


In [206]:
df_final = pd.concat([df_a_garder_1, df_a_garder_2, df_a_garder_3, df_a_garder_4])

In [207]:
df_a_travailler_1, df_a_travailler_2, df_a_travailler_3 = 0, 0, 0
df_a_garder_1, df_a_garder_2, df_a_garder_3, df_a_garder_4 = 0, 0, 0, 0

In [208]:
df_films = pd.concat([df_final, df_null])

In [209]:
df_final, df_null = 0, 0

VOTES

In [210]:
# On met de côté les valeurs inutiles

df_null5 = df_films[((df_films['vote_average'].isna()) & (df_films['averageRating'].isna()))]
df_null6 = df_films[((df_films['vote_average'] == 0) & (df_films['averageRating'] == 0))]
df_null7 = df_films[((df_films['vote_average'] == 0) & (df_films['averageRating'].isna()))]
df_null8 = df_films[((df_films['vote_average'].isna()) & (df_films['averageRating'] == 0))]

In [211]:
df_null9 = pd.concat([df_null5, df_null6, df_null7, df_null8])

In [212]:
df_null5, df_null6, df_null7, df_null8 = 0, 0, 0, 0

In [213]:
# On prend les valeurs utiles

df_ratings = df_films[~((df_films['vote_average'].isna()) & (df_films['averageRating'].isna()))]
df_ratings = df_ratings[~((df_ratings['vote_average'] == 0) & (df_ratings['averageRating'] == 0))]
df_ratings = df_ratings[~((df_ratings['vote_average'] == 0) & (df_ratings['averageRating'].isna()))]
df_ratings = df_ratings[~((df_ratings['vote_average'].isna()) & (df_ratings['averageRating'] == 0))]

In [214]:
df_ratings['vote_average'] = round(df_ratings['vote_average'], 1)

In [215]:
# On met de côté là où les 2 valeurs sont identiques

df_a_garder_1 = df_ratings[df_ratings['averageRating'] == df_ratings['vote_average']]
df_a_travailler_1 = df_ratings[~(df_ratings['averageRating'] == df_ratings['vote_average'])]

In [216]:
# On met de côté là où une valeur manque seulement

df_a_garder_2 = df_a_travailler_1[(df_a_travailler_1['averageRating'].isna()) | (df_a_travailler_1['vote_average'].isna())| (df_a_travailler_1['vote_average'] == 0) | (df_a_travailler_1['averageRating'] == 0)]
df_a_travailler_2 = df_a_travailler_1[~((df_a_travailler_1['averageRating'].isna()) | (df_a_travailler_1['vote_average'].isna())| (df_a_travailler_1['vote_average'] == 0) | (df_a_travailler_1['averageRating'] == 0))]

In [217]:
df_a_travailler_2['difference_vote'] = abs(df_a_travailler_2['averageRating'] - df_a_travailler_2['vote_average'])

/var/folders/2b/18xblx_n4lv24wz932vjckrm0000gp/T/ipykernel_4339/2546417221.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_a_travailler_2['difference_vote'] = abs(df_a_travailler_2['averageRating'] - df_a_travailler_2['vote_average'])


In [218]:
# On met de côté là où la différence de durée est négligeable

df_a_garder_3 = df_a_travailler_2[df_a_travailler_2['difference_vote'] <= 1]
df_a_travailler_3 = df_a_travailler_2[df_a_travailler_2['difference_vote'] > 1]

In [219]:
df_a_travailler_3['vote_exact'] = ((df_a_travailler_3['vote_count'] * df_a_travailler_3['vote_average']) + (df_a_travailler_3['numVotes'] * df_a_travailler_3['averageRating'])) / (df_a_travailler_3['numVotes'] + df_a_travailler_3['vote_count'])
df_a_travailler_3['vote_exact'] = round(df_a_travailler_3['vote_exact'], 1)
df_a_garder_4 = df_a_travailler_3

/var/folders/2b/18xblx_n4lv24wz932vjckrm0000gp/T/ipykernel_4339/4062215657.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_a_travailler_3['vote_exact'] = ((df_a_travailler_3['vote_count'] * df_a_travailler_3['vote_average']) + (df_a_travailler_3['numVotes'] * df_a_travailler_3['averageRating'])) / (df_a_travailler_3['numVotes'] + df_a_travailler_3['vote_count'])
/var/folders/2b/18xblx_n4lv24wz932vjckrm0000gp/T/ipykernel_4339/4062215657.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_a_travaill

In [220]:
# On crée la colonne vote_exact pour garder la moyenne des votes

df_a_garder_1['vote_exact'] = df_a_garder_1[['averageRating', 'vote_average']].max(axis = 1)
df_a_garder_2['vote_exact'] = df_a_garder_2[['averageRating', 'vote_average']].max(axis = 1)
df_a_garder_3['vote_exact'] = df_a_garder_3[['averageRating', 'vote_average']].mean(axis = 1)

/var/folders/2b/18xblx_n4lv24wz932vjckrm0000gp/T/ipykernel_4339/3653439724.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_a_garder_1['vote_exact'] = df_a_garder_1[['averageRating', 'vote_average']].max(axis = 1)
/var/folders/2b/18xblx_n4lv24wz932vjckrm0000gp/T/ipykernel_4339/3653439724.py:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_a_garder_2['vote_exact'] = df_a_garder_2[['averageRating', 'vote_average']].max(axis = 1)
/var/folders/2b/18xblx_n4lv24wz932vjckrm0000gp/T/ipykernel_4339/36534397

In [221]:
# On concatène tous les tableaux conservés

df_final = pd.concat([df_a_garder_1, df_a_garder_2, df_a_garder_3, df_a_garder_4])
df_final['arrondi_vote_exact'] = round(df_final['vote_exact'], 0)

In [222]:
df_a_travailler_1, df_a_travailler_2, df_a_travailler_3 = 0, 0, 0
df_a_garder_1, df_a_garder_2, df_a_garder_3, df_a_garder_4 = 0, 0, 0, 0

In [223]:
df_films = pd.concat([df_final, df_null9])

In [224]:
df_final, df_null9 = 0, 0

GENRE

In [225]:
df_films = df_films.rename(columns = {'genres_x' : 'genres_y', 'genres_y' : 'genres_x'})

In [226]:
df_films[df_films[['genres_y', 'genres_x']] == '\\N'] = df_films[df_films[['genres_y', 'genres_x']] == '\\N'].replace('\\N', np.nan)

/var/folders/2b/18xblx_n4lv24wz932vjckrm0000gp/T/ipykernel_4339/4290074865.py:1: FutureWarning: Downcasting behavior in `replace` is deprecated and will be removed in a future version. To retain the old behavior, explicitly call `result.infer_objects(copy=False)`. To opt-in to the future behavior, set `pd.set_option('future.no_silent_downcasting', True)`
  df_films[df_films[['genres_y', 'genres_x']] == '\\N'] = df_films[df_films[['genres_y', 'genres_x']] == '\\N'].replace('\\N', np.nan)


In [227]:
def transfo_liste(x):
  if isinstance(x, str):
    return ast.literal_eval(x)
  else:
    return x
  
df_films['genres_y'] = df_films['genres_y'].apply(transfo_liste)

In [228]:
df_films['genres_y'] = df_films['genres_y'].apply(lambda x: x if (type(x) == list) else [])

In [229]:
df_films['genres_x'] = df_films['genres_x'].str.replace(",", ", ")

In [230]:
df_films['genres_x_list'] = df_films['genres_x'].fillna('').str.split(', ')

In [231]:
df_films['genre'] = df_films.apply(lambda row: list(set(row['genres_y'] + row['genres_x_list'])),axis=1)
df_films['genre'] = df_films['genre'].apply(lambda x: x if x != [] else np.nan)

In [232]:
df_films = df_films.drop(['genres_x_list', 'genres_x', 'genres_y'], axis = 1)

TITLE

In [233]:
# df_films['title'] = df_films.apply(lambda row : row['title_y'] if (row['region'].str.contains('FR') == True) else row['title_y'].fillna(row['title_x']).fillna(row['primaryTitle']).fillna(row['original_title']).fillna(row['originalTitle']))

df_films['title'] = df_films['title_y'].fillna(df_films['title_x']).fillna(df_films['primaryTitle']).fillna(df_films['original_title']).fillna(df_films['originalTitle'])
df_films = df_films.drop(['original_title', 'title_y', 'title_x', 'primaryTitle', 'originalTitle'], axis = 1)

ADULT

In [234]:
df_films['adult'] = (df_films['isAdult'] == 1) | df_films['adult']
df_films = df_films.drop(['isAdult'], axis=1)

TCONST

In [235]:
df_films['film_id'] = df_films['imdb_id'].fillna(df_films['tconst']).fillna(df_films['titleId'])

In [236]:
df_films = df_films.drop(['imdb_id', 'tconst', 'titleId'], axis = 1)

VOTE COUNT

In [237]:
df_films['vote_count_mean'] = (df_films['vote_count'] + df_films['numVotes'])/2

In [238]:
df_films = df_films.drop(['runtime', 'release_date', 'vote_average', 'numVotes', 'vote_count', 'startYear', 'endYear', 'runtimeMinutes', 'averageRating', 'difference_runtime', 'difference_vote'], axis = 1)

FINAL LANGUAGE

In [239]:
colonnes_y = ['production_countries', 'spoken_languages', 'production_companies_country']

for cl in colonnes_y:
 
    df_films[cl] = df_films[cl].apply(transfo_liste)
    df_films[cl] = df_films[cl].apply(lambda x: x if (type(x) == list) else [])

In [240]:
df_films['original_language_x_list'] = df_films['original_language'].fillna('').str.split(', ')

In [241]:
df_films['language_x_list'] = df_films['language'].fillna('').str.split(', ')

In [243]:
df_films['final_language'] = df_films.apply(lambda row: list(set(row['production_countries'] + row['spoken_languages'] + row['production_companies_country'] + row['original_language_x_list'] + row['language_x_list'])),axis=1)

In [244]:
df_films['final_language'] = df_films['final_language'].apply(lambda x: x if x != [] else np.nan)

In [245]:
df_films['final_language'] = df_films['final_language'].apply(lambda x: ', '.join(x) if type(x) == list else x)

In [246]:
df_films = df_films.drop(['original_language', 'spoken_languages', 'language', 'original_language_x_list', 'language_x_list'], axis = 1)

In [247]:
df_films = df_films[(df_films['final_language'].isna() == True) | (df_films['final_language'] == '') | (df_films['final_language'].str.contains('fr') == True) | (df_films['final_language'].str.contains('FR') == True)  | (df_films['final_language'].str.contains('us') == True) | (df_films['final_language'].str.contains('US') == True)]

CLEAN FINAL DU MERGE ET GROUP BY

In [349]:
df_str['language_fr'] = df_str['final_language'].apply(lambda x : 1 if ('fr' in x or 'FR' in x) else 0)

In [357]:
df_str = df_str.sort_values(by = 'language_fr', ascending = False)

In [248]:
df_str = df_films.select_dtypes(exclude = 'number')
df_str_drop_dup = df_str.drop_duplicates(subset = ['film_id'], keep = 'first')

In [249]:
df_num = pd.concat([df_films['film_id'], df_films.select_dtypes(include = 'number')], axis = 1)
df_num = df_num.drop(['revenue', 'budget'], axis = 1)

In [250]:
df_revenue_budget = df_films[['film_id', 'revenue', 'budget']]
df_revenue_budget_gb = df_revenue_budget.groupby('film_id').max()

In [251]:
df_num_gb = df_num.groupby('film_id').max()

In [252]:
df_num2 = pd.merge(df_num_gb,
                   df_revenue_budget_gb,
                   how = 'outer',
                   on = 'film_id')

In [253]:
df_films = pd.merge(df_str_drop_dup,
                   df_num2,
                   on = 'film_id',
                   how = 'left')

In [254]:
df_films = df_films[df_films['status'] == 'Released']

In [255]:
df_films = df_films.drop(['isOriginalTitle', 'revenue', 'budget', 'ordering', 'types', 'attributes', 'video', 'id'], axis = 1)

In [256]:
df_films = df_films[~(df_films['vote_count_mean'] == 0)] # A voir si on le conserve
df_films = df_films[~(df_films['year_exact'] == 0)]
df_films = df_films[~(df_films['runtime_exact'].isna() == True)]
df_films['region'] = df_films['region'].fillna('Unknown')

In [257]:
df_films = df_films.rename(columns = {
    'backdrop_path' : 'backdrop_path_out_KNN',
    'homepage' : 'homepage_out_KNN',
    'overview' : 'overview_out_KNN',
    'poster_path' : 'poster_path_out_KNN',
    'film_id' : 'film_id_out_KNN',
    'title' : 'title_out_KNN',
})

CHANGEMENT GENRE

In [258]:
tous_les_genres = set()
for genres in df_films['genre']:
  tous_les_genres.update(genres)

In [259]:
tous_les_genres.remove("")

In [260]:
for genre in tous_les_genres:
  df_films[f'genre{genre}'] = df_films['genre'].apply(lambda x: genre in x)

In [261]:
df_films = df_films.drop('genre', axis = 1)

CHANGEMENT production_companies_name

In [262]:
df_films['production_companies_name2'] = df_films['production_companies_name'].apply(lambda x: x.replace("[", '').replace("]", '').replace("'", ''))

In [263]:
df_films['production_companies_name3'] = df_films['production_companies_name2'].apply(lambda x : x.split(','))

In [264]:
tous_les_genres = set()
for genres in df_films['production_companies_name3']:
  tous_les_genres.update(genres)

In [267]:
tous_les_genres.remove("")

In [269]:
for genre in tous_les_genres:
  df_films[f'production_companies_name{genre}'] = df_films['production_companies_name3'].apply(lambda x: genre in x)

TypeError: argument of type 'bool' is not iterable

In [114]:
df_films = df_films.drop(['production_companies_name', 'production_companies_name2', 'production_companies_name3'], axis = 1)

CHANGEMENT production_countries

In [118]:
# df_films['production_countries2'] = df_films['production_countries'].apply(lambda x: x.replace("['", '').replace("']", '').replace("'", ''))
# df_films['production_countries3'] = df_films['production_countries2'].apply(lambda x : x.split(','))


list

In [270]:
tous_les_genres = set()
for genres in df_films['production_countries']:
  tous_les_genres.update(genres)

In [271]:
for genre in tous_les_genres:
  df_films[f'production_countries{genre}'] = df_films['production_countries'].apply(lambda x: genre in x)

/var/folders/2b/18xblx_n4lv24wz932vjckrm0000gp/T/ipykernel_4339/3775449065.py:2: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance.  Consider joining all columns at once using pd.concat(axis=1) instead. To get a de-fragmented frame, use `newframe = frame.copy()`
  df_films[f'production_countries{genre}'] = df_films['production_countries'].apply(lambda x: genre in x)
/var/folders/2b/18xblx_n4lv24wz932vjckrm0000gp/T/ipykernel_4339/3775449065.py:2: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance.  Consider joining all columns at once using pd.concat(axis=1) instead. To get a de-fragmented frame, use `newframe = frame.copy()`
  df_films[f'production_countries{genre}'] = df_films['production_countries'].apply(lambda x: genre in x)
/var/folders/2b/18xblx_n4lv24wz932vjckrm0000gp/T/ipykernel_4339/37754490

In [124]:
df_films = df_films.drop(['production_countries'], axis = 1)

CHANGEMENT production_companies_country

In [ ]:
# df_films['production_companies_country2'] = df_films['production_companies_country'].apply(lambda x: x.replace("['", '').replace("']", '').replace("'", ''))
# df_films['production_companies_country3'] = df_films['production_companies_country2'].apply(lambda x : x.split(','))

In [307]:
df_films['production_companies_country2'] = df_films['production_companies_country'].apply(lambda x : ",".join(x).lower())

In [309]:
df_films['production_companies_country3'] = df_films['production_companies_country2'].apply(lambda x : x.split(','))

In [312]:
tous_les_genres = set()
for genres in df_films['production_companies_country3']:
  tous_les_genres.update(genres)

In [314]:
tous_les_genres.remove('')

In [315]:
for genre in tous_les_genres:
  df_films[f'production_companies_country{genre}'] = df_films['production_companies_country'].apply(lambda x: genre in x)

/var/folders/2b/18xblx_n4lv24wz932vjckrm0000gp/T/ipykernel_4339/635620503.py:2: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance.  Consider joining all columns at once using pd.concat(axis=1) instead. To get a de-fragmented frame, use `newframe = frame.copy()`
  df_films[f'production_companies_country{genre}'] = df_films['production_companies_country'].apply(lambda x: genre in x)
/var/folders/2b/18xblx_n4lv24wz932vjckrm0000gp/T/ipykernel_4339/635620503.py:2: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance.  Consider joining all columns at once using pd.concat(axis=1) instead. To get a de-fragmented frame, use `newframe = frame.copy()`
  df_films[f'production_companies_country{genre}'] = df_films['production_companies_country'].apply(lambda x: genre in x)
/var/folders/2b/18xblx_n4lv24wz932vjckrm00

In [133]:
df_films = df_films.drop('production_companies_country', axis = 1)

In [134]:
df_films = df_films.rename(columns = {'final_language' : 'final_language_out_KNN', 'tagline' : 'tagline_out_KNN', 'production_companies_country FR': 'production_companies_countryfr'})

CHANGEMENT POUR LANGUE FR

In [162]:
df_films['prod_FR'] = df_films.apply(lambda row : 1 if row['production_companies_countryFR'] == True else 0, axis = 1)

In [ ]:
# df_films['prod_FR'] = df_films.apply(lambda row : 1 if (('fr' in row['production_companies_country'] == True) or ('FR' in row['production_companies_country'])) else 0, axis = 1)
# df_films = df_films.drop(['production_companies_countryfr', 'production_companies_countryFR'], axis = 1)

CHANGEMENT POUR FILM US

In [169]:
df_films['prod_US'] = df_films.apply(lambda row : 1 if row['production_companies_countryUS'] == True else 0, axis = 1)

In [ ]:
# df_films['prod_US'] = df_films.apply(lambda row : 1 if (row['production_companies_countryUS'] == True) or (row['production_companies_country US'] == True) else 0, axis = 1)
# df_films = df_films.drop(['production_companies_countryUS', 'production_companies_country US'], axis = 1)

In [170]:
df_films = df_films.drop('production_companies_countryUS', axis = 1)

EXPORT

In [171]:
df_films.to_csv('P2_G5_films.csv.gz', index=False, compression='gzip')

KeyboardInterrupt: 